<h1><center>The Battle of Neighborhoods</center></h1>

This **capstone project** has been submitted as part of the requirements for completion of the **IBM Data Science Professional Certificate** on Coursera. In general, this project would be encompassing a series of Data Science techniques, including, but not limited to, Web Scraping (using BeautifulSoup and Requests), Data Cleaning, Data Wrangling and Machine Learning (K-Means clustering algorithm).

### Importing required libraries and packages

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!pip install geocoder
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium
import folium # map rendering library

print("Libraries imported.")

tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.
tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.
Libraries imported.


### Scraping web for neighborhood data of Berlin (WikiPedia)

In [3]:
data = requests.get("https://en.wikipedia.org/wiki/Category:Localities_of_Berlin").text
soup = BeautifulSoup(data, 'lxml')
textList = []
neighborhoodList = []

In [4]:
# append the data into the list
neighborhoodList.clear()

for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)
    
df = pd.DataFrame({"Neighborhood": neighborhoodList})
df1 = df.iloc[1:]
berlin_df = df1.reset_index(drop=True)
berlin_df.head()

,Neighborhood
0,Adlershof
1,Alt-Hohenschönhausen
2,Alt-Treptow
3,Altglienicke
4,Baumschulenweg


In [5]:
# Geographical coordinates of neighborhoods

# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Berlin, Germany'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

coords = [ get_latlng(neighborhood) for neighborhood in berlin_df["Neighborhood"].tolist() ]

df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

# merge the coordinates into the original dataframe
berlin_df['Latitude'] = df_coords['Latitude']
berlin_df['Longitude'] = df_coords['Longitude']


# check the neighborhoods and the coordinates
print(berlin_df.shape)
berlin_df.head()

(96, 3)


,Neighborhood,Latitude,Longitude
0,Adlershof,52.43779,13.54778
1,Alt-Hohenschönhausen,52.54706,13.50055
2,Alt-Treptow,52.49350,13.45711
3,Altglienicke,52.42006,13.53969
4,Baumschulenweg,52.46669,13.48840


In [6]:
# save the DataFrame as CSV file
berlin_df.to_csv("berlin_neighborhoods.csv", index=False)

#### Visualise map with neighborhoods superimposed

In [7]:
address = 'Berlin, Germany'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Berlin is {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Berlin is 52.5170365, 13.3888599.


In [8]:

# create map of Toronto using latitude and longitude values
map_berlin = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(berlin_df['Latitude'], berlin_df['Longitude'], berlin_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_berlin)  
    
map_berlin

#### Using Foursquare API

In [15]:
# The code was removed by Watson Studio for sharing.

In [16]:
radius = 2000
LIMIT = 100

venues = []
for lat, long, neighborhood in zip(berlin_df['Latitude'], berlin_df['Longitude'], berlin_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))


In [17]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(5424, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Adlershof,52.43779,13.54778,Mia Toscana,52.438327,13.549573,Italian Restaurant
1,Adlershof,52.43779,13.54778,Griechisches Restaurant Athen,52.434901,13.546680,Greek Restaurant
2,Adlershof,52.43779,13.54778,Olympia Greek Food,52.433982,13.538500,Greek Restaurant
3,Adlershof,52.43779,13.54778,Food - Taste of the world on the road,52.428982,13.537843,Food Truck
4,Adlershof,52.43779,13.54778,Trattoria Il Mondo,52.434788,13.543176,Trattoria/Osteria


In [19]:
venues_df.groupby(["Neighborhood"]).count()
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 339 uniques categories.


In [22]:
#### Analyse each neighborhood
# one hot encoding
onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot = onehot[fixed_columns]

print(onehot.shape)

grouped = onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(grouped.shape)
grouped.head()

(5424, 340)
(96, 340)


,Neighborhoods,ATM,Adult Boutique,African Restaurant,American Restaurant,Animal Shelter,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Bath House,Bathing Area,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Boat Rental,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brasserie,Breakfast Spot,Brewery,Bridge,Building,Bulgarian Restaurant,Burger Joint,Burrito Place,Bus Stop,Business Service,Butcher,Cafeteria,Café,Campground,Canal,Canal Lock,Candy Store,Capitol Building,Castle,Caucasian Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Gym,College Rec Center,Comedy Club,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Credit Union,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Currywurst Joint,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium,Forest,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Hockey Field,Hockey Rink,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Kumpir Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Light Rail Station,Liquor Store,Lottery Retailer,Lounge,Market,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Venue,Nature Preserve,Neighborhood,New American Restaurant,Nightclub,Noodle House,Nudist Beach,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoor Supply Store,Paintball Field,Pakistani Restaurant,Palace,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Piano Bar,Pide Place,Pie Shop,Pier,Piercing Parlor,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Post Office,Print Shop,Pub,Record Shop,Recreation Center,Rental Car Location,Rest Area,Restaurant,River,Road,Rock Climbing Spot,Rock Club,Roof Deck,Rugby Stadium,Russian Restaurant,Salon / Barbershop,Sandwich Place,Sauna / Steam Room,Scandinavian Restaurant,Scenic Lookout,Schnitzel Restaurant,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Squash Court,Stables,Stadium,Stationery Store,Steakhouse,Street 

In [27]:
len(grouped[grouped["Restaurant"] > 0])
berlin_rest = grouped[["Neighborhoods","Restaurant"]]
berlin_rest.head()

,Neighborhoods,Restaurant
0,Adlershof,0.021277
1,Alt-Hohenschönhausen,0.000000
2,Alt-Treptow,0.000000
3,Altglienicke,0.033333
4,Baumschulenweg,0.000000
